In [508]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import json
import requests
import folium
from folium.plugins import MarkerCluster


In [509]:
response = requests.get("https://ckan0.cf.opendata.inter.prod-toronto.ca")
response


<Response [200]>

In [510]:
import requests
import pandas as pd

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "cultural-hotspot-points-of-interest"}
package_info = requests.get(url, params=params).json()

# Check if the package exists and contains resources
if package_info["success"]:
    resources = package_info["result"]["resources"]
    for resource in resources:
        if resource["datastore_active"]:
            resource_id = resource["id"]
            # Fetch all records in CSV format
            csv_url = f"{base_url}/datastore/dump/{resource_id}"
            csv_data = requests.get(csv_url).text
            # Create a DataFrame from the CSV data
            df = pd.read_csv(io.StringIO(csv_data))
            # Display the DataFrame
            display(df.head())  # Use display() to show the DataFrame in Jupyter Notebook
else:
    print("Package not found or no active resources available.")


,_id,LoopsGuide,Loop,TourNum,OrderNum,LoopTourName,LoopTourURL,TourLabel,SiteName,Neighbourhood,TourType,Duration,EntryType,Interests,DirectionsTransit,DirectionsCar,Description,Address,ExternalLink,ImageCredit,ImageCreditExternalLink,ImageAltText,ImageURL,ThumbURL,ImageOrientation,test1,test2,test3,ObjectId,geometry
0,1,York,2,2,7.0,The Future in Mount Dennis,NaN,NaN,Up To This Moment (2019),Mount Dennis,NaN,NaN,Destination,Art,NaN,NaN,Two pieces of public art will be integrated in...,3222 Eglinton Ave. W.,NaN,NaN,NaN,NaN,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,Portrait,NaN,NaN,NaN,1,"{""type"": ""Point"", ""coordinates"": [-79.48458, 4..."
1,2,York,2,2,8.0,The Future in Mount Dennis,NaN,NaN,Untitled (2019),Mount Dennis,NaN,NaN,Destination,"Art, History",NaN,NaN,The second piece of public art that will be in...,3222 Eglinton Ave. W.,NaN,NaN,NaN,NaN,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,Portrait,NaN,NaN,NaN,2,"{""type"": ""Point"", ""coordinates"": [-79.48487, 4..."
2,3,York,2,2,12.0,The Future in Mount Dennis,NaN,NaN,John Dennis,Mount Dennis,NaN,NaN,Did you know?,History,NaN,NaN,"Mount Dennis is named for John Dennis, a shipb...",Weston Rd. and Eglinton Ave.,NaN,NaN,NaN,NaN,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,Portrait,NaN,NaN,NaN,3,"{""type"": ""Point"", ""coordinates"": [-79.48888, 4..."
3,4,York,2,2,11.0,The Future in Mount Dennis,NaN,NaN,Mount Dennis Library,Mount Dennis,NaN,NaN,Destination,"Community, History",NaN,NaN,Libraries in the Mount Dennis area date back t...,1123 Weston Rd.,https://www.torontopubliclibrary.ca/mountdennis/,Toronto Public Library,NaN,The two-story Mount Dennis Library featuring a...,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,Landscape,NaN,NaN,NaN,4,"{""type"": ""Point"", ""coordinates"": [-79.48833, 4..."
4,5,York,2,2,9.0,The Future in Mount Dennis,NaN,NaN,Black Creek at Weston Rd.,Mount Dennis,NaN,NaN,Destination,"Nature, History",NaN,NaN,Toronto's geography was shaped by repeated gla...,Humber Blvd. N. and Weston Rd.,NaN,NaN,NaN,A section of Black Creek running through a con...,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,https://contrib0.wp.intra.dev-toronto.ca/ext/e...,Portrait,NaN,NaN,NaN,5,"{""type"": ""Point"", ""coordinates"": [-79.48012, 4..."


In [539]:
import pandas as pd

# Assuming df is your DataFrame

columns_to_drop = ['Loop','_id','TourNum','OrderNum','LoopTourName', 
                   'LoopTourURL','TourLabel','ImageCredit', 'DirectionsCar','TourType','LoopsGuide','Duration','ImageCreditExternalLink','ImageURL','ThumbURL','ImageOrientation','test1','test2','test3','ObjectId']

# Check if columns exist in the DataFrame before attempting to drop them
columns_to_drop = [col for col in columns_to_drop if col in df.columns]

# Drop the specified columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)

# Print DataFrame columns to verify their names
print(df.columns)


Index(['AREA_ID', 'DATE_EFFECTIVE', 'DATE_EXPIRY', 'AREA_ATTR_ID',
       'AREA_TYPE_ID', 'PARENT_AREA_ID', 'AREA_TYPE', 'AREA_CLASS_ID',
       'AREA_CLASS', 'AREA_SHORT_CODE', 'AREA_LONG_CODE', 'AREA_NAME',
       'AREA_DESC', 'FEATURE_CODE', 'FEATURE_CODE_DESC', 'TRANS_ID_CREATE',
       'TRANS_ID_EXPIRE', 'OBJECTID', 'geometry'],
      dtype='object')


In [512]:
# Assuming 'df' is your DataFrame
# Filter the DataFrame based on values in the 'Entry type' column
filtered_df = df[df['EntryType'].isin(['Tour description','Did you know?','Did you know','Explore more'])]

# Display the filtered DataFrame
filtered_df.head()


,SiteName,Neighbourhood,EntryType,Interests,DirectionsTransit,Description,Address,ExternalLink,ImageAltText,geometry
2,John Dennis,Mount Dennis,Did you know?,History,NaN,"Mount Dennis is named for John Dennis, a shipb...",Weston Rd. and Eglinton Ave.,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-79.48888, 4..."
10,Trethewey Airfield,Mount Dennis,Explore more,History,NaN,The Trethewey Airfield was Toronto's first air...,59 Hearst Cir.,NaN,A black line illustration of a domed Trethewey...,"{""type"": ""Point"", ""coordinates"": [-79.4997787,..."
21,The Weston Historical Society,Weston,Did you know?,History,NaN,"The Weston Historical Society, an affiliate to...","1901 Weston Rd., Unit F",NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-79.51608, 4..."
22,Raymore Bridge and Plaque,Weston,Explore more,History,NaN,"The original bridge, along with many homes on ...","Raymore Park, entrance at Bellevue Cres. And H...",https://www.toronto.ca/data/parks/prd/faciliti...,NaN,"{""type"": ""Point"", ""coordinates"": [-79.514619, ..."
23,Weston Village BIA,Weston,Did you know?,Community,NaN,"The Weston Village BIA, established in 1979, i...",4 John St.,https://www.westonvillagebia.com/,NaN,"{""type"": ""Point"", ""coordinates"": [-79.5177, 43..."


In [513]:
import geopandas as gpd
from shapely.geometry import Point
import json

# Assuming 'filtered_df' is your filtered DataFrame with a 'geometry' column containing Point data

# Define a function to parse the JSON string and extract the coordinates
def parse_geometry(geom_str):
    geom_dict = json.loads(geom_str)
    return Point(geom_dict['coordinates'][0], geom_dict['coordinates'][1])

# Apply the parse_geometry function to each row of the 'geometry' column
geometry = filtered_df['geometry'].apply(parse_geometry)

# Create a GeoDataFrame from the DataFrame and the geometry column
gdf = gpd.GeoDataFrame(filtered_df, geometry=geometry)

# Now 'gdf' is a GeoDataFrame with geometries extracted from the 'geometry' column


In [514]:
# Now 'gdf' is a GeoDataFrame with the original data
gdf


,SiteName,Neighbourhood,EntryType,Interests,DirectionsTransit,Description,Address,ExternalLink,ImageAltText,geometry
2,John Dennis,Mount Dennis,Did you know?,History,NaN,"Mount Dennis is named for John Dennis, a shipb...",Weston Rd. and Eglinton Ave.,NaN,NaN,POINT (-79.48888 43.68686)
10,Trethewey Airfield,Mount Dennis,Explore more,History,NaN,The Trethewey Airfield was Toronto's first air...,59 Hearst Cir.,NaN,A black line illustration of a domed Trethewey...,POINT (-79.49978 43.69937)
21,The Weston Historical Society,Weston,Did you know?,History,NaN,"The Weston Historical Society, an affiliate to...","1901 Weston Rd., Unit F",NaN,NaN,POINT (-79.51608 43.70052)
22,Raymore Bridge and Plaque,Weston,Explore more,History,NaN,"The original bridge, along with many homes on ...","Raymore Park, entrance at Bellevue Cres. And H...",https://www.toronto.ca/data/parks/prd/faciliti...,NaN,POINT (-79.51462 43.69688)
23,Weston Village BIA,Weston,Did you know?,Community,NaN,"The Weston Village BIA, established in 1979, i...",4 John St.,https://www.westonvillagebia.com/,NaN,POINT (-79.51770 43.70097)
40,Weston CNR Bridge,Weston,Did you know?,"Architecture, History",NaN,"Just above North End Parkette, the Canadian Na...",Weston Rd. and Humberview Cres.,NaN,NaN,POINT (-79.53316 43.70697)
43,Weston Mount Dennis Community Place Hub,Weston,Did you know?,Community,NaN,The Weston Mount Dennis Community Place Hub is...,1765 Weston Rd.,NaN,NaN,POINT (-79.51238 43.69929)
52,The Weston Farmers' Market,Weston,Did you know?,Community,NaN,The Weston Farmers’ Market (Saturdays from May...,34 John St.,https://www.westonvillagebia.com/market/,NaN,POINT (-79.51788 43.70194)
56,Water Mural,Weston,Explore more,Art,NaN,"This 8,000-square-foot mural was a collaborati...","Jane St., Highway 400 Underpass",NaN,NaN,POINT (-79.50598 43.71119)
63,Flemingdon Park,"Milne Hollow, East Don River",Did you know,Nature,NaN,This area is home to the Korean Canadian Cultu...,1133 Leslie St.,https://www.toronto.ca/data/parks/prd/faciliti...,An illustrated black line drawing of modern bu...,POINT (-79.34835 43.72642)


In [515]:
import folium

# Construct the Mapbox style URL
mapbox_style_url =mapbox_style_url = "https://api.mapbox.com/styles/v1/sgtkumar/clt7n1sep004901qrflti09vt/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2d0a3VtYXIiLCJhIjoiY2xzdnFjMmpyMmQybjJqcHM2eHhyajlyZSJ9.aQrxL6j9uSGZseuBzaB8TQ"

# Create a Folium map object
m = folium.Map(location=[43.7, -79.4], zoom_start=8, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Save the map to an HTML file
m

In [538]:
import folium

# Construct the Mapbox style URL
mapbox_style_url = "https://api.mapbox.com/styles/v1/sgtkumar/clt7n1sep004901qrflti09vt/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2d0a3VtYXIiLCJhIjoiY2xzdnFjMmpyMmQybjJqcHM2eHhyajlyZSJ9.aQrxL6j9uSGZseuBzaB8TQ"
# Create a Folium map object centered around Toronto
m = folium.Map(location=[43.7, -79.4], zoom_start=10, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Plot the GeoDataFrame on the map
for idx, row in gdf.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup=row['SiteName']).add_to(m)

# Save the map to an HTML file
m.save("map_with_points.html")

# Display the map
m


In [543]:
import pandas as pd

# URL to the CSV file
csv_file_url = "/Users/siddarthrathinam/Documents/Documents/ENVS456/labs/City Wards Data - 4326.csv"

# Load the data using pandas
df_new = pd.read_csv(csv_file_url)

# Print the DataFrame
print(df_new)


    _id  AREA_ID  DATE_EFFECTIVE  DATE_EXPIRY  AREA_ATTR_ID  AREA_TYPE_ID  \
0     1  2457740             NaN          NaN      25993196           528   
1     2  2457739             NaN          NaN      25993195           528   
2     3  2457738             NaN          NaN      25993194           528   
3     4  2457737             NaN          NaN      25993193           528   
4     5  2457736             NaN          NaN      25993192           528   
5     6  2457735             NaN          NaN      25993191           528   
6     7  2457734             NaN          NaN      25993190           528   
7     8  2457733             NaN          NaN      25993189           528   
8     9  2457732             NaN          NaN      25993188           528   
9    10  2457731             NaN          NaN      25993187           528   
10   11  2457730             NaN          NaN      25993186           528   
11   12  2457729             NaN          NaN      25993185           528   

In [533]:
csv_file_url = "/Users/siddarthrathinam/Documents/Documents/ENVS456/labs/City Wards Data - 4326.csv"
df_new = pd.read_csv(csv_file_url)
df_new

,_id,AREA_ID,DATE_EFFECTIVE,DATE_EXPIRY,AREA_ATTR_ID,AREA_TYPE_ID,PARENT_AREA_ID,AREA_TYPE,AREA_CLASS_ID,AREA_CLASS,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,FEATURE_CODE,FEATURE_CODE_DESC,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,geometry
0,1,2457740,NaN,NaN,25993196,528,NaN,CITW,NaN,NaN,7,7,Humber River-Black Creek,Humber River-Black Creek (7),NaN,NaN,279754,-1,17344785,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
1,2,2457739,NaN,NaN,25993195,528,NaN,CITW,NaN,NaN,6,6,York Centre,York Centre (6),NaN,NaN,279754,-1,17344801,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,3,2457738,NaN,NaN,25993194,528,NaN,CITW,NaN,NaN,18,18,Willowdale,Willowdale (18),NaN,NaN,279754,-1,17344817,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
3,4,2457737,NaN,NaN,25993193,528,NaN,CITW,NaN,NaN,11,11,University-Rosedale,University-Rosedale (11),NaN,NaN,279754,-1,17344833,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
4,5,2457736,NaN,NaN,25993192,528,NaN,CITW,NaN,NaN,19,19,Beaches-East York,Beaches-East York (19),NaN,NaN,279754,-1,17344849,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
5,6,2457735,NaN,NaN,25993191,528,NaN,CITW,NaN,NaN,20,20,Scarborough Southwest,Scarborough Southwest (20),NaN,NaN,279754,-1,17344865,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
6,7,2457734,NaN,NaN,25993190,528,NaN,CITW,NaN,NaN,25,25,Scarborough-Rouge Park,Scarborough-Rouge Park (25),NaN,NaN,279754,-1,17344881,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
7,8,2457733,NaN,NaN,25993189,528,NaN,CITW,NaN,NaN,23,23,Scarborough North,Scarborough North (23),NaN,NaN,279754,-1,17344897,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
8,9,2457732,NaN,NaN,25993188,528,NaN,CITW,NaN,NaN,24,24,Scarborough-Guildwood,Scarborough-Guildwood (24),NaN,NaN,279754,-1,17344913,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
9,10,2457731,NaN,NaN,25993187,528,NaN,CITW,NaN,NaN,21,21,Scarborough Centre,Scarborough Centre (21),NaN,NaN,279754,-1,17344929,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."


KeyError: 'MultiPolygon'

In [544]:
# Set CRS to EPSG:4326 (WGS84)
gdf.crs = 'EPSG:4326'

# Construct the Mapbox style URL
mapbox_style_url = "https://api.mapbox.com/styles/v1/sgtkumar/clt7n1sep004901qrflti09vt/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2d0a3VtYXIiLCJhIjoiY2xzdnFjMmpyMmQybjJqcHM2eHhyajlyZSJ9.aQrxL6j9uSGZseuBzaB8TQ"

# Create a Folium map object
m = folium.Map(location=[43.7, -79.4], zoom_start=8, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Plot the polygons
folium.GeoJson(gdf,
                name='geojson',
                style_function=lambda feature: {
                    'color': 'black',
                    'fillColor': 'transparent',
                    'weight': 1
                },
                highlight_function=lambda x: {'weight': 3, 'color': 'blue'}
               ).add_to(m)

# Add AREA_NAME as text labels
for idx, row in gdf.iterrows():
    folium.Marker(location=[row.geometry.centroid.y, row.geometry.centroid.x],
                  icon=folium.DivIcon(html=f'<div style="font-size: 10pt">{row["AREA_NAME"]}</div>')
                 ).add_to(m)

# Display the map
m

KeyError: 'AREA_NAME'

In [540]:
import folium
import geopandas as gpd

# Assuming your DataFrame is already loaded into df_new

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df_new, geometry='geometry')

# Set CRS to EPSG:4326 (WGS84)
gdf.crs = 'EPSG:4326'

# Construct the Mapbox style URL
mapbox_style_url = "https://api.mapbox.com/styles/v1/sgtkumar/clt7n1sep004901qrflti09vt/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2d0a3VtYXIiLCJhIjoiY2xzdnFjMmpyMmQybjJqcHM2eHhyajlyZSJ9.aQrxL6j9uSGZseuBzaB8TQ"

# Create a Folium map object
m = folium.Map(location=[43.7, -79.4], zoom_start=8, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Plot the polygons and add AREA_NAME as text labels
folium.GeoJson(gdf,
               name='geojson',
               style_function=lambda feature: {
                   'color': 'black',
                   'fillColor': 'transparent',
                   'weight': 1
               },
               highlight_function=lambda x: {'weight': 3, 'color': 'blue'},
               tooltip=folium.features.GeoJsonTooltip(fields=['AREA_NAME']),
               popup=folium.features.GeoJsonPopup(fields=['AREA_NAME'])
              ).add_to(m)

# Display the map
m


TypeError: Input must be valid geometry objects: {'type': 'MultiPolygon', 'coordinates': [[[[-79.4910534106931, 43.7635021528307], [-79.4900820250133, 43.7598284098823], [-79.4899806571011, 43.7593255316339], [-79.4896582169971, 43.7580571182177], [-79.4894943462708, 43.7574480826525], [-79.4892217914919, 43.7563581298315], [-79.4887625068059, 43.7545124715302], [-79.488581248007, 43.7537566590354], [-79.4884793050092, 43.7533281017588], [-79.4883438009846, 43.752758501149], [-79.4881735629245, 43.7520237517077], [-79.4894699236512, 43.7518257800638], [-79.4895185511648, 43.7518929064883], [-79.4895719022077, 43.7519325526566], [-79.4896539891458, 43.7519621560644], [-79.4897034917939, 43.7519966621367], [-79.4897747485622, 43.7520394962462], [-79.4898354501373, 43.7520846966884], [-79.4898929219094, 43.7521378358713], [-79.489952408517, 43.7521724957376], [-79.4900878008623, 43.7522993884784], [-79.4902335368447, 43.7524311425883], [-79.4903424150251, 43.7525039535446], [-79.490422733

In [541]:
import folium
import geopandas as gpd

# Assuming your DataFrame is already loaded into df_new for polygons
# and df_pointers for pointers

# Create a GeoDataFrame from the DataFrame for polygons
gdf_polygons = gpd.GeoDataFrame(df_new, geometry='geometry')

# Set CRS to EPSG:4326 (WGS84) for polygons
gdf_polygons.crs = 'EPSG:4326'

# Construct the Mapbox style URL
mapbox_style_url = "https://api.mapbox.com/styles/v1/sgtkumar/clt7n1sep004901qrflti09vt/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2d0a3VtYXIiLCJhIjoiY2xzdnFjMmpyMmQybjJqcHM2eHhyajlyZSJ9.aQrxL6j9uSGZseuBzaB8TQ"

# Create a Folium map object
m = folium.Map(location=[43.7, -79.4], zoom_start=8, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Plot the polygons and add AREA_NAME as text labels
folium.GeoJson(gdf_polygons,
               name='polygons',
               style_function=lambda feature: {
                   'color': 'black',
                   'fillColor': 'transparent',
                   'weight': 1
               },
               highlight_function=lambda x: {'weight': 3, 'color': 'blue'},
               tooltip=folium.features.GeoJsonTooltip(fields=['AREA_NAME']),
               popup=folium.features.GeoJsonPopup(fields=['AREA_NAME'])
              ).add_to(m)

# Create a GeoDataFrame from the DataFrame for pointers
gdf_pointers = gpd.GeoDataFrame(df_pointers, geometry=gpd.points_from_xy(df_pointers.longitude, df_pointers.latitude))

# Set CRS to EPSG:4326 (WGS84) for pointers
gdf_pointers.crs = 'EPSG:4326'

# Iterate over the GeoDataFrame and add points as markers to the map
for idx, row in gdf_pointers.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup=row['name']).add_to(m)

# Save the map as an HTML file
m.save("map_with_polygons_and_pointers.html")

# Display the map
m


TypeError: Input must be valid geometry objects: {'type': 'MultiPolygon', 'coordinates': [[[[-79.4910534106931, 43.7635021528307], [-79.4900820250133, 43.7598284098823], [-79.4899806571011, 43.7593255316339], [-79.4896582169971, 43.7580571182177], [-79.4894943462708, 43.7574480826525], [-79.4892217914919, 43.7563581298315], [-79.4887625068059, 43.7545124715302], [-79.488581248007, 43.7537566590354], [-79.4884793050092, 43.7533281017588], [-79.4883438009846, 43.752758501149], [-79.4881735629245, 43.7520237517077], [-79.4894699236512, 43.7518257800638], [-79.4895185511648, 43.7518929064883], [-79.4895719022077, 43.7519325526566], [-79.4896539891458, 43.7519621560644], [-79.4897034917939, 43.7519966621367], [-79.4897747485622, 43.7520394962462], [-79.4898354501373, 43.7520846966884], [-79.4898929219094, 43.7521378358713], [-79.489952408517, 43.7521724957376], [-79.4900878008623, 43.7522993884784], [-79.4902335368447, 43.7524311425883], [-79.4903424150251, 43.7525039535446], [-79.490422733

In [ ]:
import folium
import geopandas as gpd

# Assuming your DataFrame is already loaded into df_new

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df_new, geometry='geometry')

# Set CRS to EPSG:4326 (WGS84)
gdf.crs = 'EPSG:4326'

# Construct the Mapbox style URL
mapbox_style_url = "https://api.mapbox.com/styles/v1/sgtkumar/clt7n1sep004901qrflti09vt/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2d0a3VtYXIiLCJhIjoiY2xzdnFjMmpyMmQybjJqcHM2eHhyajlyZSJ9.aQrxL6j9uSGZseuBzaB8TQ"

# Create a Folium map object
m = folium.Map(location=[43.7, -79.4], zoom_start=8, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Plot the polygons
folium.GeoJson(gdf,
                name='geojson',
                style_function=lambda feature: {
                    'color': 'black',
                    'fillColor': 'transparent',
                    'weight': 1
                },
                highlight_function=lambda x: {'weight': 3, 'color': 'blue'}
               ).add_to(m)

# Add AREA_NAME as text labels
for idx, row in gdf.iterrows():
    folium.Marker(location=[row.geometry.centroid.y, row.geometry.centroid.x],
                  icon=folium.DivIcon(html=f'<div style="font-size: 10pt">{row["AREA_NAME"]}</div>')
                 ).add_to(m)

# Display the map
m


In [545]:
import folium

# Create a Folium map object
m = folium.Map(location=[43.7, -79.4], zoom_start=8, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Plot the polygons and add AREA_NAME as text labels
folium.GeoJson(gdf,
               name='geojson',
               style_function=lambda feature: {
                   'color': 'black',
                   'fillColor': 'transparent',
                   'weight': 1
               },
               highlight_function=lambda x: {'weight': 3, 'color': 'blue'},
               tooltip=folium.features.GeoJsonTooltip(fields=['AREA_NAME'])
              ).add_to(m)

# Iterate over the GeoDataFrame and add points as markers to the map
for idx, row in gdf.iterrows():
    folium.Marker(location=[row.geometry.centroid.y, row.geometry.centroid.x], 
                  icon=folium.DivIcon(html=f'<div style="font-size: 10pt">{row["AREA_NAME"]}</div>')
                 ).add_to(m)

# Display the map
m


KeyError: 'AREA_NAME'

In [526]:
import pandas as pd
import geopandas as gpd
import folium

# URL to the CSV file
csv_file_url = "/Users/siddarthrathinam/Documents/Documents/ENVS456/labs/City Wards Data - 4326.csv"

# Load the data using pandas
df = pd.read_csv(csv_file_url)

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Construct the Mapbox style URL
mapbox_style_url = "https://api.mapbox.com/styles/v1/sgtkumar/clt7n1sep004901qrflti09vt/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2d0a3VtYXIiLCJhIjoiY2xzdnFjMmpyMmQybjJqcHM2eHhyajlyZSJ9.aQrxL6j9uSGZseuBzaB8TQ"

# Create a Folium map object
m = folium.Map(location=[43.7, -79.4], zoom_start=8, tiles=mapbox_style_url, attr="Mapbox", control_scale=True)

# Iterate over the GeoDataFrame and add points as markers to the map
for idx, row in gdf.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup=row['Address']).add_to(m)

# Save the map to an HTML file
m.save("map_with_points.html")


AttributeError: 'DataFrame' object has no attribute 'longitude'